In [1]:
import pandas as pd
import numpy as np
from cobra  import Model, Reaction, Metabolite

In [2]:
model_name = 'iMAC868'
sheets = ['reactions', 'metabolites', 'genes', 'reaction bounds', 'std. transformed Gibbs energies', 'proteomics data']

In [3]:
file = 'iMAC868.xlsx'
# file = 'iMAC868_GSM.xlsx'
loaded = [pd.read_excel(f'./Data/iMAC868/{file}', sheet_name=s) for s in sheets];

In [4]:
saved = [loaded[i].to_csv('./Data/iMAC868/iMAC868_' + sheets[i] + '.csv', index=False) for i in range(len(loaded))];

In [5]:
sheets_dict = dict(enumerate(sheets))
sheets_dict = {v: k for k, v in sheets_dict.items()}

In [6]:
loaded[sheets_dict['reactions']]

,Abbreviation,Name,Reaction,GPR,Subsystem,Reversible,Lower bound,Upper bound,Objective
0,ADSL1,adenylosuccinate lyase,dcamp[c] <==> amp[c] + fum[c],MA3971,Alanine and Aspartate Metabolism,0,0.0,1000,0
1,ADSS,adenylosuccinate synthase,asp-L[c] + gtp[c] + imp[c] <==> dcamp[c] + gdp...,(MA1919 or MA4118),Alanine and Aspartate Metabolism,0,0.0,1000,0
2,ALATA_L,L-alanine transaminase,akg[c] + ala-L[c] <==> glu-L[c] + pyr[c],(MA0636 or MA0925 or MA1385 or MA1712 or MA1819),Alanine and Aspartate Metabolism,1,-1000.0,1000,0
3,ALATRS,Alanyl-tRNA synthetase,ala-L[c] + atp[c] + trnaala[c] <==> amp[c] + p...,(MA0194 or MA2014),Alanine and Aspartate Metabolism,0,0.0,1000,0
4,APAT2,beta-alanine:2-oxoglutarate aminotransferase,akg[c] + ala-B[c] <==> glu-L[c] + msa[c],MA2859,Alanine and Aspartate Metabolism,0,0.0,1000,0
...,...,...,...,...,...,...,...,...,...
840,RNF_rev,Rnf functioning in reverse,(2) mphenh2[c] + (2) fdox[c] + (0) h[c] <==> (...,NaN,Reverse Pathway,0,0.0,0,0
841,ETR_MPR,hypothetical electron transport through methan...,mphenh2[c] + (0) h[c] <==> mphen[c] + eae[c] +...,NaN,Reverse Pathway,0,0.0,0,0
842,HYP_HYD1,hypothetical hydrogenase,com[c] + cob[c] <==> hsfd[c] + h2[c],NaN,Reverse Pathway,0,0.0,0,0
843,HYP_HYD2,hypothetical hydrogenase 2,f420-2h2[c] <==> f420-2[c] + h2[c] + h[c],NaN,Reverse Pathway,0,0.0,0,0


### Load reactions properties

In [7]:
reactions_abbreviations     = loaded[0]['Abbreviation'].tolist()
reaction_names              = loaded[0]['Name'].tolist()
reaction_subsystem          = loaded[0]['Subsystem'].tolist()
lower_bound                 = loaded[0]['Lower bound'].tolist()
upper_bound                 = loaded[0]['Upper bound'].tolist()
objective                   = loaded[0]['Objective'].tolist()
reaction_arr                = loaded[0]['Reaction'].tolist()
gene_reaction_rules         = loaded[0]['GPR'].tolist()

### Load metabolite properties

In [8]:
met_abbreviations   = loaded[1]['Abbreviation'].tolist()
met_names           = loaded[1]['Name'].tolist()
met_formula         = loaded[1]['Formula (charged)'].tolist()
met_charge          = loaded[1]['Charge'].tolist()
met_KEGG_id         = loaded[1]['KEGG ID'].tolist()

In [9]:
# met_formula = [str(formula).replace(' ', '') for formula in met_formula]

### Load genes properties

In [10]:
gene_loci           = loaded[2]['Loci'].tolist()
gene_annotation     = loaded[2]['Annotation'].tolist()
gene_ec_number      = loaded[2]['EC #'].tolist()
gene_protein_abbreviation = loaded[2]['Protein abbreviation'].tolist()

### Load reaction bounds

In [11]:
rxn_bound_reaction           = loaded[3]['Reaction'].tolist()
LB_native_substrates         = loaded[3]['LB (growth with native substrates)'].tolist()
UB_native_substrates         = loaded[3]['UB (growth with native substrates)'].tolist()
LB_growth_with_methane       = loaded[3]['LB (growth with methane)'].tolist()
UB_growth_with_methane       = loaded[3]['UB (growth with methane)'].tolist()

In [12]:
model = Model('iMAC868')
media = 'metahane'

compartment = [met_abbreviations[i].split('[')[1].split(']')[0] for i in range(len(met_abbreviations))]
splitted_rxns = [(reaction_arr[i].replace('+',' ').replace('<', ' ').replace('==', ' ').replace('>', ' ')).split(' ') for i in range(len(reaction_arr))]
splitted_rxns = [[item for item in splitted_rxns[i] if item != ''] for i in range(len(splitted_rxns))]

gene_reaction_rules = [str(gene_reaction_rules[i]).replace('(', '').replace(')' , '') for i in range(len(gene_reaction_rules))]

met_lenght = range(len(met_abbreviations))
met_per_rxn = [{Metabolite(met_abbreviations[i], formula=met_formula[i], name=met_names[i], compartment=compartment[i]) : met_charge[i] for i in met_lenght for ridx in range(len(rxn)) if str(rxn[ridx]).replace(' ', '') == str(met_abbreviations[i]).replace(' ', '')} for rxn in splitted_rxns]

for i in range(len(reactions_abbreviations)):

    reaction             = Reaction(reactions_abbreviations[i])

    reaction.name        = reaction_names[i]
    reaction.subsystem   = reaction_subsystem[i]
    if media == 'native':
        reaction.lower_bound = LB_native_substrates[i]
        reaction.upper_bound = UB_native_substrates[i]
    elif media == 'methane':
        reaction.lower_bound = LB_growth_with_methane[i]
        reaction.upper_bound = UB_growth_with_methane[i]
    reaction.gene_reaction_rule =  gene_reaction_rules[i]

    reaction.add_metabolites(met_per_rxn[i])
    model.add_reaction(reaction)
    
print(len(model.metabolites))

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
526


In [13]:
met_arr_in_model =[model.metabolites[i].id for i in range(len(model.metabolites))]

In [14]:
not_in_model = [met_abbreviations[i] for i in range(len(met_abbreviations)) if met_abbreviations[i] not in met_arr_in_model];

In [15]:
objective_name = [reactions_abbreviations[i] for i in range(len(objective)) if objective[i] == 1][0]

In [16]:
model.objective = objective_name
model.objective.expression

1.0*overall - 1.0*overall_reverse_00a0e

In [17]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions[:5]:
    print(f"{x.id} : {x.reaction}")

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites[:5]:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes[:5]:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
ADSL1 : 2 amp[c] + 4 dcamp[c] + 2 fum[c] --> 
ADSS : asp-L[c] + 4 dcamp[c] + 3 gdp[c] + 4 gtp[c] + 2 imp[c] + 2 pi[c] --> h[c]
ALATA_L : 2 akg[c] + glu-L[c] + pyr[c] --> 
ALATRS : 2 amp[c] + 4 atp[c] + 3 ppi[c] --> alatrna[c]
APAT2 : 2 akg[c] + glu-L[c] + msa[c] --> 

Metabolites
-----------
   amp[c] : C10H12N5O7P
 dcamp[c] : C14H14N5O11P
   fum[c] : C4H2O4
 asp-L[c] : C4H6NO4
   gdp[c] : C10H12N5O11P2

Genes
-----
MA3971 is associated with reactions: {ADSL2, ADSL1}
MA1919 is associated with reactions: {ADSS}
MA4118 is associated with reactions: {ADSS}
MA1819 is associated with reactions: {TRPTA, PHETA1, ASPTA, CYSTA, ALATA_L, TYRTA, LCYSTAT}
MA0636 is associated with reactions: {TRPTA, PHETA1, ASPTA, CYSTA, ALATA_L, TYRTA, LCYSTAT}


### Model Validation

In [18]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model

with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

SBML errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


(<Model iMAC868 at 0x7fd730a46df0>,
 {'COBRA_CHECK': ["Metabolite 'lac-D[c]' formula 'C3H5O3 ' not alphanumeric",
                  "Metabolite 'lgt-S[c]' formula 'C13H20N3O8S ' not "
                  'alphanumeric'],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': ['E0 (Error): SBML component consistency (fbc, L334); Chemical '
                 'formula must be string; The value of attribute '
                 "'fbc:chemicalFormula' on the SBML <species> object must be "
                 'set to a string consisting only of atomic names or user '
                 'defined compounds and their occurrence.\n'
                 'Reference: L3V1 Fbc V3 Section 3.4\n'
                 " Encountered ' ' when expecting a capital letter.The "
                 "chemicalFormula 'C3H5O3 ' has incorrect syntax.\n",
                 'E1 (Error): SBML component consistency (fbc, L335); Chemical '
                 'formula must be string; The value of attribute '
     

In [19]:
model

Name,iMAC868
Memory address,0x07fd730f9a5e0
Number of metabolites,526
Number of reactions,845
Number of groups,0
Objective expression,1.0*overall - 1.0*overall_reverse_00a0e
Compartments,"c, e"


In [20]:
solution = model.optimize()
solution

,fluxes,reduced_costs
ADSL1,0.0,0.0
ADSS,0.0,0.0
ALATA_L,0.0,0.0
ALATRS,0.0,-2.0
APAT2,0.0,0.0
...,...,...
RNF_rev,0.0,-0.0
ETR_MPR,0.0,-0.0
HYP_HYD1,0.0,0.0
HYP_HYD2,0.0,0.0


In [21]:
met_arr_b = set(model.reactions) - set(model.boundary)
len(met_arr_b)

796

In [22]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [23]:
model.objective.direction

'max'